Ziel: CSV mit Aktivitätsdaten einlesen und die gesamte Dauer (s) sowie die gesamte Distanz (m) pro Jahr und Aktivitätstyp berechnen.

In [1]:
import pandas as pd
from pathlib import Path
import plotly.express as px

In [2]:
csv_path = Path('activities.csv')

df = pd.read_csv(csv_path)

if not pd.api.types.is_datetime64_any_dtype(df.get('start_time')):
    df['start_time'] = pd.to_datetime(df['start_time'], errors='coerce')
    print('Konvertierung durchgeführt: start_time -> datetime (errors=\'coerce\')')

print('Dtypes nach parse:', df.dtypes.to_dict())
if df['start_time'].isna().any():
    n_bad = df['start_time'].isna().sum()
    print(f"Warnung: {n_bad} start_time-Wert(e) konnten nicht geparst werden und sind NaT")

if pd.api.types.is_datetime64_any_dtype(df['start_time']):
    df['year'] = df['start_time'].dt.year
else:
    raise ValueError('start_time ist nicht datetimelike; Konvertierung fehlgeschlagen')

agg = (
    df.groupby(['year', 'activity_type'], as_index=False)
      .agg(total_duration_s=('duration_s', 'sum'),
           total_distance_m=('distance_m', 'sum'))
)

agg = agg.sort_values(['year', 'activity_type']).reset_index(drop=True)
print(agg.to_string(index=False))

Konvertierung durchgeführt: start_time -> datetime (errors='coerce')
Dtypes nach parse: {'start_time': dtype('<M8[ns]'), 'duration_s': dtype('float64'), 'distance_m': dtype('float64'), 'activity_type': dtype('O')}
Warnung: 1 start_time-Wert(e) konnten nicht geparst werden und sind NaT
  year  activity_type  total_duration_s  total_distance_m
2014.0        running        460421.473        1351262.64
2015.0        running        475468.069        1378825.00
2016.0        running        795870.098        2354310.45
2017.0        cycling        228403.000        1192122.05
2017.0         hiking        337597.000         417926.72
2017.0 mountaineering        275491.000         163613.30
2017.0        running        843071.276        2479471.46
2017.0       swimming        116400.000           7500.00
2018.0        cycling       1390843.000        3423594.39
2018.0         hiking         57419.000         115926.67
2018.0  rock_climbing         76800.000              0.00
2018.0        runn

In [4]:
for activity in agg['activity_type'].unique():
    activity_data = agg[agg['activity_type'] == activity]
    
    fig = px.bar(
        activity_data,
        x='year',
        y='total_distance_m',
        title=f'Distanz pro Jahr - {activity}',
        labels={'total_distance_m': 'Distanz (m)', 'year': 'Jahr'}
    )
    
    # Zeige das Diagramm
    fig.show()